# Fruit recognition

## Chargement des modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as im
import sklearn as sk
import glob
import os

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

## Chargement des données (train)

In [ ]:
tabImg=pd.DataFrame()
categories = ['Apple Braeburn', 'Apple Granny Smith', 'Apricot', 'Avocado', 'Banana', 'Blueberry', 'Cactus fruit', 'Cantaloupe', 'Cherry', 'Clementine', 'Corn', 'Cucumber Ripe', 'Grape Blue', 'Kiwi', 'Lemon', 'Limes', 'Mango', 'Onion White', 'Orange', 'Papaya', 'Passion Fruit', 'Peach', 'Pear', 'Pepper Green', 'Pepper Red', 'Pineapple', 'Plum', 'Pomegranate', 'Potato Red', 'Raspberry', 'Strawberry', 'Tomato', 'Watermelon']
label = []
root_dir=os.getcwd()+"\\data\\train\\"

for filename in glob.iglob(root_dir+'**\\*.jpg',recursive=True):
    img=im.imread(filename)
    img=img.reshape(img.shape[0]*img.shape[1]*img.shape[2])
    
    fruit_name=filename.split('\\')[-2]
    
    label.append(categories.index(fruit_name))
    tabImg=tabImg.append(pd.Series(img),ignore_index=True)
tabImg['label']=label

In [ ]:
tabImg.shape

In [ ]:
tabImg.to_csv('train.csv',index=False)

In [ ]:
np.save('train.npy', tabImg)

In [ ]:
tabImg.head(5)

## Récupération des données (train)

In [2]:
#tab=pd.read_csv('train.csv')
tab = np.load('train.npy', allow_pickle=True)

In [3]:
print(tab.shape)
tab=pd.DataFrame(tab)

(16854, 30001)


In [4]:
tab.head(5)

,0,1,2,3,4,5,6,7,8,9,...,29991,29992,29993,29994,29995,29996,29997,29998,29999,30000
0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,0.0
1,255.0,255.0,246.0,253.0,255.0,247.0,253.0,255.0,249.0,251.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,0.0
2,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,0.0
3,255.0,255.0,251.0,255.0,255.0,253.0,255.0,254.0,252.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,0.0
4,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,0.0


In [5]:
tab=tab.sample(frac=1) # Pour mélanger
tab.head(5)

,0,1,2,3,4,5,6,7,8,9,...,29991,29992,29993,29994,29995,29996,29997,29998,29999,30000
5252,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,10.0
3354,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,7.0
13229,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,25.0
15328,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,30.0
14928,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,29.0


In [6]:
tab.isna().sum()

0        0
1        0
2        0
3        0
4        0
        ..
29996    0
29997    0
29998    0
29999    0
30000    0
Length: 30001, dtype: int64

# Traitement des données et modèles

In [7]:
X=tab.iloc[:,:-1]
print(X.shape)
X.head(5)

(16854, 30000)


,0,1,2,3,4,5,6,7,8,9,...,29990,29991,29992,29993,29994,29995,29996,29997,29998,29999
5252,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
3354,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
13229,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
15328,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0
14928,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,...,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0,255.0


In [8]:
Y=tab.iloc[:,-1]
print(Y.shape)
Y.head(5)

(16854,)


5252     10.0
3354      7.0
13229    25.0
15328    30.0
14928    29.0
Name: 30000, dtype: float64

# KKN

In [9]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y,test_size=0.25, random_state=0)

In [10]:
print(Xtrain.shape)
print(Xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(12640, 30000)
(4214, 30000)
(12640,)
(4214,)


In [11]:
KKN = KNeighborsClassifier()
clf=KKN.fit(Xtrain,ytrain)

ypredit=clf.predict(Xtest)

In [12]:
print(accuracy_score(ytest,ypredit))

0.9988134788799241


accuracy : 0.9988134788799241

In [13]:
print(metrics.confusion_matrix(ytest, ypredit))

[[119   0   0 ...   0   0   0]
 [  0 121   0 ...   0   0   0]
 [  0   0 115 ...   0   0   0]
 ...
 [  0   0   0 ... 122   0   0]
 [  0   0   0 ...   0 190   0]
 [  0   0   0 ...   0   0 110]]


# SVM


In [14]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y,test_size=0.25, random_state=0)

In [16]:
clf=svm.SVC(gamma="auto")
clf.fit(Xtrain,ytrain)

ypredit=clf.predict(Xtest)

In [17]:
accuracy_score(ytest,ypredit)

0.060512577123872806

accuracy : 0.060512577123872806

In [18]:
print(metrics.confusion_matrix(ytest,ypredit))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Matrice : 
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

# Decision Tree

In [19]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, Y,test_size=0.25, random_state=0)

In [20]:
Arbre_decision = DecisionTreeClassifier(random_state=0, max_depth=10)
clf = Arbre_decision.fit(Xtrain, ytrain)

ypredit=clf.predict(Xtest)

In [21]:
accuracy_score(ytest,ypredit)

0.7520170859041291

accuracy : 0.7520170859041291

In [22]:
print(metrics.confusion_matrix(ytest,ypredit))

[[ 26   3   0 ...   0   0   0]
 [  0 119   0 ...   0   0   0]
 [  1   0  95 ...   0   0   0]
 ...
 [  0   1   1 ...  40   0   0]
 [  0   0   0 ...   0 180   0]
 [  0   0   0 ...   0   0  83]]
